In [1]:
import IPython.display
import sys
import json
from rdflib import *
import glob
import os
import re
from SPARQLWrapper import SPARQLWrapper, JSON, BASIC
import xml.etree.ElementTree as ET

In [2]:
def loadAuthFile():
    with open('graphdb_auth.txt') as f:
        auth =  f.readlines()
        auth = [x.replace('\n','') for x in auth]
    return auth

In [3]:
def getSPARQLResults(queryString, auth):
    result = None
    sparql = SPARQLWrapper(auth[0] + "KGRC4SI_20230714_r2")
    sparql.setHTTPAuth(BASIC)
    sparql.setCredentials(auth[1], auth[2])
    sparql.setQuery(queryString)
    sparql.setReturnFormat(JSON)
    try :
        json = sparql.query().convert()
        result = json
    except  Exception as e:
        print(e)
    return result

In [4]:
def getActivities():
    auth = loadAuthFile()
    query = """
            PREFIX ho: <http://www.owl-ontologies.com/VirtualHome.owl#>
            PREFIX ex: <http://kgrc4si.home.kg/virtualhome2kg/instance/>
            PREFIX vh2kg: <http://kgrc4si.home.kg/virtualhome2kg/ontology/>
            SELECT DISTINCT ?s 
            WHERE { 
                ?s vh2kg:virtualHome ?scene . 
            }
        """
    result = getSPARQLResults(query, auth)
    activity_list = []
    for row in result["results"]["bindings"]:
        activity_link = row['s']['value']
        activity_list.append(activity_link.replace('http://kgrc4si.home.kg/virtualhome2kg/instance/', ''))
    return activity_list

In [5]:
def getFirstSituation(activity_name):
    auth = loadAuthFile()
    query = """
            prefix ex: <http://kgrc4si.home.kg/virtualhome2kg/instance/> 
            prefix ho: <http://www.owl-ontologies.com/VirtualHome.owl#> 
            prefix ns1: <http://purl.org/dc/terms/> 
            prefix owl: <http://www.w3.org/2002/07/owl#> 
            prefix rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#> 
            prefix rdfs: <http://www.w3.org/2000/01/rdf-schema#> 
            prefix time: <http://www.w3.org/2006/time#> 
            prefix vh2kg: <http://kgrc4si.home.kg/virtualhome2kg/ontology/> 
            prefix vh2kg-an: <http://kgrc4si.home.kg/virtualhome2kg/ontology/action/> 
            prefix x3do: <https://www.web3d.org/specifications/X3dOntology4.0#> 
            prefix xsd: <http://www.w3.org/2001/XMLSchema#> 
            select distinct ?firstEvent ?firstSituation ?state ?objectType ?objectLabel ?objectId ?BCX ?BCY ?BCZ ?BSX ?BSY ?BSZ where { 
                ?activity vh2kg:hasEvent ?firstEvent.
                ?firstEvent a vh2kg:StartEvent;
                            vh2kg:situationBeforeEvent ?firstSituation.
                ?state vh2kg:partOf ?firstSituation;
                       vh2kg:bbox ?shape;
                       vh2kg:isStateOf ?object.
                ?shape x3do:bboxCenter ?bboxCenter;
                       x3do:bboxSize ?bboxSize;
                       a x3do:Shape.
                ?bboxCenter rdf:first ?BCX.
                ?bboxCenter rdf:rest ?BCYb .
                ?BCYb rdf:first ?BCY ;
                      rdf:rest ?BCZb .
                ?BCZb rdf:first ?BCZ .
                ?bboxSize rdf:first ?BSX.
                ?bboxSize rdf:rest ?BSYb .
                ?BSYb rdf:first ?BSY  ;
                      rdf:rest ?BSZb .
                ?BSZb rdf:first ?BSZ .
                ?object a ?objectType;
                        rdfs:label ?objectLabel;
                        ns1:identifier ?objectId.

                filter(?activity = ex:"""+activity_name+""")
            }
        """
    result = getSPARQLResults(query, auth)
    return result

In [6]:
def getNextSituation(index, activity):
    auth = loadAuthFile()
    query = """
            prefix ex: <http://kgrc4si.home.kg/virtualhome2kg/instance/> 
            prefix ho: <http://www.owl-ontologies.com/VirtualHome.owl#> 
            prefix ns1: <http://purl.org/dc/terms/> 
            prefix owl: <http://www.w3.org/2002/07/owl#> 
            prefix rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#> 
            prefix rdfs: <http://www.w3.org/2000/01/rdf-schema#> 
            prefix time: <http://www.w3.org/2006/time#> 
            prefix vh2kg: <http://kgrc4si.home.kg/virtualhome2kg/ontology/> 
            prefix vh2kg-an: <http://kgrc4si.home.kg/virtualhome2kg/ontology/action/> 
            prefix x3do: <https://www.web3d.org/specifications/X3dOntology4.0#> 
            prefix xsd: <http://www.w3.org/2001/XMLSchema#> 
            select distinct ?nextSituation ?state ?objectType ?objectLabel ?objectId ?BCX ?BCY ?BCZ where { 
                ?situation vh2kg:nextSituation ?nextSituation.
                ?state vh2kg:partOf ?nextSituation ;
                       vh2kg:bbox ?shape;
                       vh2kg:isStateOf ?object.
                ?shape x3do:bboxCenter ?bboxCenter;
                       x3do:bboxSize ?bboxSize;
                       a x3do:Shape.
                ?bboxCenter rdf:first ?BCX.
                ?bboxCenter rdf:rest ?BCYb .
                ?BCYb rdf:first ?BCY ;
                      rdf:rest ?BCZb .
                ?BCZb rdf:first ?BCZ .
                ?object a ?objectType;
                        rdfs:label ?objectLabel;
                        ns1:identifier ?objectId.
                filter(?situation = ex:home_situation"""+index+"_"+activity+""")
            }
        """
    result = getSPARQLResults(query, auth)
    return result

In [7]:
def getSituationCount(activity):
    auth = loadAuthFile()
    query = """
            PREFIX ex: <http://kgrc4si.home.kg/virtualhome2kg/instance/>
            PREFIX vh2kg: <http://kgrc4si.home.kg/virtualhome2kg/ontology/>
            PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
            SELECT DISTINCT (count(?event) AS ?cnt)
            WHERE { 
                ex:"""+activity+""" vh2kg:hasEvent ?event .
            }
        """
    result = getSPARQLResults(query, auth)
    count = result["results"]["bindings"][0]['cnt']['value']
    return int(count)

Generate HTML files

In [27]:
def transform_maker(key, object):
    transform = ET.Element('transform', {'DEF': key})
    shape = shape_maker(object)
    transform.append(shape)

    return transform


def shape_maker(object):
    shape = ET.Element('shape')
    appearance = ET.SubElement(shape, 'appearance')
    # characterの場合だけ赤色
    if object["objectType"] == "character":
        material = ET.SubElement(appearance, 'material', {'diffuseColor': '1 0 0', 'transparency': '0'})
    else: 
        material = ET.SubElement(appearance, 'material', {'diffuseColor': '0 1 0', 'transparency': '0.9'})
    material.text = ' '
    a = object["bboxSize"]
    size = str(float(a[0]))+","+str(float(a[1]))+","+str(float(a[2]))
    box = ET.SubElement(shape, 'box', {'size': size, 'solid': 'false'})
    box.text = " "

    return shape

def timeSensor_maker(num):
    timeSensor = ET.Element('timeSensor', {'DEF': 'time', 'cycleInterval': str(num), 'loop': 'true'})
    timeSensor.text = " "

    return timeSensor


def PositionInterpolator_maker(key, object):
    keys = ""
    for k in object["key"]:
        keys = keys + str(k) + " "
    keyValue = ""
    for kv in object["keyValue"]:
        keyValue = keyValue + str(float(kv[0]))+" "+str(float(kv[1]))+" "+str(float(kv[2])) + "  "
    positionInterpolator = ET.Element('PositionInterpolator', {'DEF': "move"+key, 'key': keys, 'keyValue': keyValue})
    positionInterpolator.text = " "

    return positionInterpolator


def Route_maker(key):
    route1 = ET.Element('Route', {'fromNode': 'time', 'fromField': 'fraction_changed', 'toNode': "move"+key, 'toField': 'set_fraction'})
    route1.text = " "
    route2 = ET.Element('Route', {'fromNode': "move"+key, 'fromField': 'value_changed', 'toNode': key, 'toField': 'translation'}) 
    route2.text = " "
    routes = [route1, route2]

    return routes

for activity in getActivities():
    try:
        html = ET.Element('html')
        # headべた書き
        head = ET.SubElement(html, 'head')
        meta = ET.SubElement(
            head, 'meta', {'http-equiv': 'X-UA-Compatible', 'content': 'IE=edge'})
        title = ET.SubElement(head, 'title')
        title.text = "KGRC-RDFのbboxを描いてみた"
        script = ET.SubElement(head, 'script', {
                               'type': 'text/javascript', 'src': 'https://www.x3dom.org/download/x3dom.js'})
        script.text = ' '
        link = ET.SubElement(head, 'link', {
                             'rel': 'stylesheet', 'type': 'text/css', 'href': 'https://www.x3dom.org/download/x3dom.css'})
        link.text = ' '

        # bodyとhtml要素
        body = ET.SubElement(html, 'body')
        sectionLabel = ET.SubElement(body, 'h1')
        sectionLabel.text = "virtualhome2kg-"+activity[:-7]+".ttlのbboxを描いてみる"
        descriptionLabel = ET.SubElement(body, 'p')
        descriptionLabel.text = "操作方法は下の方にあります．マウスでくるくる回せます．キャラクターだけ赤色，他は透明な緑色です．動いてますが，動画と時間はあっていません．バウンディングボックスのサイズは初期値のままです．キャラクタがかがんだり方向転換していたりしても変わりません．"

        # ここから中身
        x3d = ET.SubElement(body, 'x3d', {'width': '1920px', 'height': '1080px'})
        scene = ET.SubElement(x3d, 'scene')
        ET.SubElement(scene, 'environment', {'smallFeatureCulling': 'true'}).text = " "

        # first situation
        # sizeをもらわないといけない．
        firstSituation = getFirstSituation(activity)

        objectDict = {}

        for row in firstSituation["results"]["bindings"]:
            objectDict[row["objectLabel"]["value"] + row["objectId"]["value"]] = {
                "bboxSize": (row["BSX"]["value"], row["BSY"]["value"], row["BSZ"]["value"]),
                "keyValue": [(row["BCX"]["value"], row["BCY"]["value"], row["BCZ"]["value"])],  # ここはappendで追加する．
                "key": [0],  # ここもappendでkeyのlengthを追加する．
                "objectType": row["objectLabel"]["value"],
            }


        # 残りのsituation
        # sizeは一個目そのまま．繰り返しで足していく．
        count = getSituationCount(activity)
        isAlreadyCopy = False
        for index in range(count):
            nextSituation = getNextSituation(str(index), activity)
            for row in nextSituation["results"]["bindings"]:
                # エージェントのbboxが上から下へ下がっている状態を直すために。。
                if row["objectLabel"]["value"] == 'character':
                    if isAlreadyCopy is False:
                        for key in objectDict:
                            if key.startswith('character'):
                                x,y,z = objectDict[key]['keyValue'][0]
                                objectDict[key]['keyValue'][0] = (x, row["BCY"]["value"], z)
                                isAlreadyCopy = True
                objectDict[row["objectLabel"]["value"] + row["objectId"]["value"]]["keyValue"].append((row["BCX"]["value"], row["BCY"]["value"], row["BCZ"]["value"]))
                objectDict[row["objectLabel"]["value"] + row["objectId"]["value"]]["key"].append(len(objectDict[row["objectLabel"]["value"] + row["objectId"]["value"]]["key"]))

        # pprint.pprint(objectDict)

        # nodeを作ります．
        # タイムセンサーが先
        timeSensor = timeSensor_maker(4)
        scene.append(timeSensor)
        # 物は後
        for o in objectDict:
            transform = transform_maker(o, objectDict[o])
            scene.append(transform)
            positionInterpolator = PositionInterpolator_maker(o, objectDict[o])
            scene.append(positionInterpolator)
            routes = Route_maker(o)
            for route in routes:
                scene.append(route)

        ## 操作方法の説明
        ET.SubElement(body, "h4").text = "操作方法"
        table = ET.SubElement(body, 'table')
        tr = ET.SubElement(table, 'tr')
        ET.SubElement(tr, 'th').text = "機能"
        ET.SubElement(tr, 'th').text = "マウスボタン"
        tbody = ET.SubElement(table, 'tbody')
        tr1 = ET.SubElement(tbody, 'tr')
        ET.SubElement(tr1, 'td').text = "回転"
        ET.SubElement(tr1, 'td').text = "左 / 左 + Shift"
        tr2 = ET.SubElement(tbody, 'tr')
        ET.SubElement(tr2, 'td').text = "パン"
        ET.SubElement(tr2, 'td').text = "真ん中 / 左 + Ctrl"
        tr3 = ET.SubElement(tbody, 'tr')
        ET.SubElement(tr3, 'td').text = "ズーム"
        ET.SubElement(tr3, 'td').text = "右 / ホイール / 左 + Alt"
        tr4 = ET.SubElement(tbody, 'tr')
        ET.SubElement(tr4, 'td').text = "描画対象が真ん中に来るようにする"
        ET.SubElement(tr4, 'td').text = "左をダブルクリック"
        tr5 = ET.SubElement(tbody, 'tr')
        ET.SubElement(tr5, 'td').text = "ビューを初期値に戻す"
        ET.SubElement(tr5, 'td').text = "キーボードのA"
        tr6 = ET.SubElement(tbody, 'tr')
        ET.SubElement(tr6, 'td').text = "小さいものも表示する"
        ET.SubElement(tr6, 'td').text = "キーボードのS"

        tree = ET.ElementTree(html)
        file_path = "virtualhome2kg/"
        if os.path.exists(file_path) == False:
            os.mkdir(file_path)
        tree.write('virtualhome2kg/virtualhome2kg-'+activity+'.html', encoding='utf-8')
    except Exception as ex:
        print(ex)